## DON'T RERUN! USE THIS AS A RECORD OF THE PROCESS.

In [1]:
import flywheel
import pandas as pd

In [2]:
fw = flywheel.Client()
#project = fw.get('5e1cec8e3a9694005b6d12ab') # HUP3TLegacy
project = fw.lookup('pennftdcenter/HUP3TLegacy')
excel = pd.read_csv('../data_collection/originals/Controls-Prodromals_10-1.csv')

Get list of subject/sessions from excel sheet

In [3]:
excel_sid_list = list(excel['INDDID'])
excel_ses_list = list(excel['MRIDateFormatted'])
print(len(excel_sid_list))
# convert sid list to string 
for i in range(len(excel_sid_list)):
    if str(excel_sid_list[i]).endswith('0'): # get rid of the trailing .0s. just need the integer (in string form)
        excel_sid_list[i] = str(int(excel_sid_list[i]))
    else:
        excel_sid_list[i] = str(excel_sid_list[i])
        

459


Create list of subjects and list of sessions in Flywheel project

In [4]:
fw_sid_list = []
for subject in project.subjects():
    #print(subject.label)
    fw_sid_list.append(subject.label)

fw_ses_list = []
for ses in project.sessions():
    #print(subject.label)
    fw_ses_list.append(ses.label)
    
print("N subjects: " + str(len(fw_sid_list)))
print("N sessions: " + str(len(fw_ses_list)))

N subjects: 2197
N sessions: 3799


Get sessions in collection

In [5]:
collection = fw.get('5eb5081448fe1b1e5792a7a9')
collection_id = '5eb5081448fe1b1e5792a7a9'
sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]

Loop through sessions in FTDC, check if FW subject label in the excel list. If so, check if the corresponding FW session is the same as the corresponding session in the excel sheet. Loop through the sessions acquisitions. Tally the subjects, the sessions, and the acquisitions. 

Some sessions have more than one T1, hence the higher number of acquisitions.

In [6]:
sessions_list = []
ses_container_list = []
subs_list = []
acq_list = []
for ses in project.sessions():
    sub = fw.get(ses.parents['subject'])
    fw_ses_label = ses.label.split('-')[0]
    if sub.label in excel_sid_list:
        subs_list.append(sub.label)
        row = excel.loc[excel['INDDID']==float(sub.label)]
        excel_ses_label = list(row['MRIDateFormatted'])[0]
        if fw_ses_label == str(excel_ses_label):
            sessions_list.append(ses.label)
            ses_container_list.append(ses)
            for acq in ses.acquisitions():
#                 print(acq.files[0]['classification'])
                has_mes = 'Measurement' in acq.files[0]['classification']
                if has_mes:
                    has_t1 = 'T1' in acq.files[0]['classification']['Measurement']
                else:
                    continue
#                 print('has measurements: ' + str(has_mes))
#                 print('has T1: ' + str(has_t1))
                if has_mes and has_t1:
                    acq_list.append(acq.label)
            
print("N subs: " + str(len(subs_list)))
print("N sessions: " + str(len(sessions_list)))
print("N acquisitions: " + str(len(acq_list)))

N subs: 856
N sessions: 440
N acquisitions: 682


The subjects number is much higher than the number of sessions, because subjects have multiple sessions and we started by looping through all the sessions. Looking at the set of unique values in the subjects list should give us a number similar to the number of sessions.

In [7]:
len(set(subs_list))

443

Close...which subjects in Excel and Flywheel don't have a Flywheel session corresponding to the subject's session in Excel?

In [63]:
sessions_subs = [0]*len(ses_container_list) # create a list of the subject names for sessions in sessions_list
for i in range(len(ses_container_list)):
    sub_id = ses_container_list[i].parents["subject"]
    sub_label = fw.get(sub_id).label
    sessions_subs[i] = sub_label

len(sessions_subs)

440

But how many are unique?

In [65]:
len(set(sessions_subs))

437

In [64]:
diff_set = set(subs_list) - set(sessions_subs)
diff_set

{'102622', '106555', '106954', '109758', '123119', '124847'}

Compared Excel and Flywheel and noticed that the session dates don't quite match up for these subjects. We can add these to the collection manually.

There are also some duplicates in the sessions_list:

In [37]:
print(len(sessions_list))
print(len(set(sessions_list)))

440
438


In [38]:
import collections
print([item for item, count in collections.Counter(sessions_list).items() if count > 1])

['20140621-1627', '20190430-1259']


We can remove these from the collection manually.

Why is len(sessions_subs) = 440, but there are only 437 in the set?
Check for duplicates:

In [66]:
print([item for item, count in collections.Counter(sessions_subs).items() if count > 1])

['118252', '120841', '124249']


118252 has 20140621-1627 and 120841 has 20190430-1259.

120841 has two different sessions on same day (03-16-2016). But both are empty in Flywheel. Will manually remove.

In [67]:
def Diff(li1, li2): 
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2] 
    return li_dif

diff = Diff(list(set(subs_list)), sessions_subs)
print(diff)
print(len(set(diff)))

['109758', '106954', '123119', '102622', '124847', '106555']
6


Double check that the sessions are in the excel file.

In [68]:
# Make the lists the same format (the dates as strings)
excel_ses_list2 = [0] * len(excel_ses_list)
for i in range(len(excel_ses_list)):
    excel_ses_list2[i] = str(excel_ses_list[i]) # turn into string
    
sessions_list2 = [0] * len(sessions_list)
for i in range(len(sessions_list)):
    sessions_list2[i] = sessions_list[i].split('-')[0] # remove the four trailing digits after the dash

In [69]:
# Create a list tallying when the assembled sessions are in the excel list
bool_list = []
for ses in sessions_list2:
    if ses in excel_ses_list2:
        bool_list.append(True)
print(len(bool_list)) # should be 440 because they should all be in the excel sheet

440


### Add the sessions to collection

In [70]:
for container in ses_container_list:
    collection.add_sessions(container.id)

Convert the container list to a list of labels

In [73]:
print(len(ses_container_list))

440


In [76]:
ses_container_labels = []
for ses in ses_container_list:
    ses_container_labels.append(ses.label)
print(len(ses_container_labels))

440


In [77]:
len(set(ses_container_labels))

438

In [91]:
print([item for item, count in collections.Counter(ses_container_labels).items() if count > 1])

['20140621-1627', '20190430-1259']


### Identify missing sessions / subjects

In [79]:
collection_sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)] 
print(len(collection_sessions))

861


In [128]:
ses_in_collect = []
for ses in collection_sessions:
    project = fw.get(ses.parents["project"])
    if 'HUP3TLegacy' in project.label:
        ses_in_collect.append(ses.label)
print(len(ses_in_collect))
print('---')

435
---


In [88]:
set(ses_container_labels) - set(ses_in_collect)

{'20110728-1250', '20160316-1459', '20160316-1604'}

In [93]:
ses_in_collect2 = [0]*len(ses_in_collect)
for i in range(len(ses_in_collect)):
    ses_in_collect2[i] = ses_in_collect[i].split('-')[0] # remove the four trailing digits after the dash

In [102]:
print(len(set(ses_in_collect2)))

384


So there are "duplicates" but just because we truncated the four trailing digits (aka the time of day). Some sessions happened on the same day. Which is why we can't use sets to determine the number missing.

In [95]:
diff_list = list(set(excel_ses_list2) - set(ses_in_collect2))
print(len(diff_list))
diff_list

18


['20091120',
 '20090811',
 '20121019',
 '20191220',
 '20191217',
 '20171212',
 '20050816',
 '20081013',
 '20081001',
 '20191118',
 '20200117',
 '20060402',
 '20191115',
 '20160316',
 '20191029',
 '20191015',
 '20051018',
 '20200113']

Instead, we allow for "duplicates":

In [104]:
diff_list2 = Diff(excel_ses_list2, ses_in_collect2)
print(len(diff_list2))
diff_list2

21


['20090811',
 '20051018',
 '20051018',
 '20060402',
 '20081001',
 '20091120',
 '20081013',
 '20191217',
 '20191217',
 '20191029',
 '20191029',
 '20191115',
 '20191015',
 '20191118',
 '20200117',
 '20050816',
 '20121019',
 '20191220',
 '20200113',
 '20171212',
 '20160316']

Kind of confusing. What we really want to look at is subjects, though. Let's do a similar comparison and try to reconstruct the 24 missing.

In [112]:
sub_in_collect = []
for ses in collection_sessions:
    project = fw.get(ses.parents["project"])
    if 'HUP3TLegacy' in project.label:
        sub = fw.get(ses.parents["subject"])
        sub_in_collect.append(sub.label)
print(len(sub_in_collect))
print(len(set(sub_in_collect)))
print('---')

435
435
---


In [115]:
sub_diff_list = Diff(excel_sid_list, sub_in_collect)
print(len(sub_diff_list))
print(len(set(sub_diff_list)))
sub_diff_list

24
24


['100979',
 '102622',
 '105114',
 '106069',
 '106555',
 '106954',
 '109177',
 '109758',
 '112002',
 '114783',
 '125678',
 '125677',
 '125530',
 '125542',
 '125277',
 '124847',
 '125278',
 '123119',
 '108783.1',
 '107837.1',
 '125781',
 '125333',
 '122700',
 '120841']

This makes more sense...

From Nik:
+ 102622 -- take flywheel session 20090810 (this was likely a chead upload typo)
+ 106555 -- take flywheel session 20090406 (same as ^)
+ 106954 -- take flywheel session 20060402 (same as ^^)
+ 123119 -- the 20200117 date is in the HUP6 project but is a little confusing as there is two sessions on that date. The one with timestamp 08:19 contains the ABCD_T1_vNav_passive scan. 
+ 109758 -- the 20081002 scan is usable. Unsure why the 2009 scan didn't make it to HUP3TLegacy however
+ 124847 -- the 20190916 scan is usable. The other one has not been uploaded to flywheel yet it is still on the PMACS cluster 

This will bring the number of subjects in the collection to 441 (435 + 6).

From Nik:
For the last 18 just on quick scan through any scans post 12/10/2019 should be in the HUP6 project. Scans in between ~10/1/2019 - 12/10/2019 are not yet on flywheel.

Now I will add scans post 12/10/2019 from HUP6 project. Just going to search remaining subjects and check the date.

Now get the sessions in the collection again:

In [118]:
collection_sessions2 = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)] 
print(len(collection_sessions2))

871


In [126]:
sub_in_collect2 = []
for ses in collection_sessions2:
    project = fw.get(ses.parents["project"])
    if 'HUP3TLegacy'in project.label  or 'HUP6' in project.label: # IMPT: OR HUP6
        sub = fw.get(ses.parents["subject"])
        sub_in_collect2.append(sub.label)
print(len(sub_in_collect2))
print(len(set(sub_in_collect2)))
print('---')

445
445
---


In [127]:
sub_diff_list2 = Diff(excel_sid_list, sub_in_collect2)
print(len(sub_diff_list2))
print(len(set(sub_diff_list2)))
sub_diff_list2

14
14


['100979',
 '105114',
 '106069',
 '109177',
 '112002',
 '114783',
 '125530',
 '125542',
 '125277',
 '125278',
 '108783.1',
 '107837.1',
 '122700',
 '120841']

Added 125277, session 20200130.